# 🛠️ Pokročilé používanie nástrojov s GitHub modelmi (.NET)

## 📋 Ciele učenia

Tento notebook demonštruje integračné vzory nástrojov na podnikovej úrovni pomocou Microsoft Agent Framework v .NET s GitHub modelmi. Naučíte sa vytvárať sofistikovaných agentov s viacerými špecializovanými nástrojmi, využívajúc silné typovanie C# a podnikové funkcie .NET.

**Pokročilé schopnosti nástrojov, ktoré si osvojíte:**
- 🔧 **Architektúra viacerých nástrojov**: Vytváranie agentov s viacerými špecializovanými schopnosťami
- 🎯 **Typovo bezpečné vykonávanie nástrojov**: Využívanie validácie počas kompilácie v C#
- 📊 **Podnikové vzory nástrojov**: Návrh nástrojov pripravených na produkciu a správa chýb
- 🔗 **Kompozícia nástrojov**: Kombinovanie nástrojov pre komplexné obchodné procesy

## 🎯 Výhody architektúry nástrojov v .NET

### Funkcie podnikových nástrojov
- **Validácia počas kompilácie**: Silné typovanie zaručuje správnosť parametrov nástrojov
- **Dependency Injection**: Integrácia IoC kontajnera na správu nástrojov
- **Async/Await vzory**: Nezablokované vykonávanie nástrojov s riadením zdrojov
- **Štruktúrované logovanie**: Integrované logovanie na monitorovanie vykonávania nástrojov

### Vzory pripravené na produkciu
- **Správa výnimiek**: Komplexná správa chýb s typovanými výnimkami
- **Správa zdrojov**: Správne vzory na uvoľňovanie zdrojov a správu pamäte
- **Monitorovanie výkonu**: Integrované metriky a ukazovatele výkonu
- **Správa konfigurácie**: Typovo bezpečná konfigurácia s validáciou

## 🔧 Technická architektúra

### Základné komponenty nástrojov v .NET
- **Microsoft.Extensions.AI**: Jednotná abstrakčná vrstva nástrojov
- **Microsoft.Agents.AI**: Orchestrácia nástrojov na podnikovej úrovni
- **Integrácia GitHub modelov**: Výkonný API klient s poolingom pripojení

### Pipeline vykonávania nástrojov
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Kategórie nástrojov a vzory

### 1. **Nástroje na spracovanie dát**
- **Validácia vstupov**: Silné typovanie s anotáciami dát
- **Transformačné operácie**: Typovo bezpečná konverzia a formátovanie dát
- **Obchodná logika**: Nástroje na výpočty a analýzy špecifické pre doménu
- **Formátovanie výstupov**: Generovanie štruktúrovaných odpovedí

### 2. **Integračné nástroje**
- **API konektory**: Integrácia RESTful služieb pomocou HttpClient
- **Nástroje pre databázy**: Integrácia Entity Framework na prístup k dátam
- **Operácie so súbormi**: Bezpečné operácie so súborovým systémom s validáciou
- **Externé služby**: Vzory integrácie služieb tretích strán

### 3. **Pomocné nástroje**
- **Spracovanie textu**: Manipulácia a formátovanie reťazcov
- **Operácie s dátumom/časom**: Výpočty dátumu/času s ohľadom na kultúru
- **Matematické nástroje**: Presné výpočty a štatistické operácie
- **Nástroje na validáciu**: Validácia obchodných pravidiel a overovanie dát

## ⚙️ Predpoklady a nastavenie

**Vývojové prostredie:**
- .NET 9.0 SDK alebo vyšší
- Visual Studio 2022 alebo VS Code s rozšírením pre C#
- Prístup k GitHub Models API

**Požadované NuGet balíčky:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Konfigurácia prostredia (.env súbor):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Pripravení vytvárať agentov na podnikovej úrovni s výkonnými, typovo bezpečnými schopnosťami nástrojov v .NET? Poďme navrhnúť profesionálne riešenia! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj by sa mal považovať pôvodný dokument v jeho pôvodnom jazyku. Pre dôležité informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
